# This notebook will serve as the interactive notebook for leveraging the SNAP Graph Processing Tool using the subprocess library. 
- Allows for easier bulk processing of many interferograms, which is needed for non-zero closure phase correction (InSAR triplets)
- Allows for easier bulk processing of custom GRD-like products (will produce both calibrate and uncalibrated backscatter data for classification)

In [1]:
# import dependencies
import os
from time import perf_counter
import subprocess
import shutil

# Functions

In [2]:
def project_dir(cwd, project_name):
    work_dir = os.path.join(cwd, project_name)
    os.makedirs(work_dir, exist_ok=True)
    data_dir = os.path.join(work_dir,'0_initial')
    os.makedirs(data_dir, exist_ok=True)

    return work_dir, data_dir

def get_triplets(slc_list):
    """
    this program will generate a dictionary with keys triplet_n, where n is the triplet stack
    each triplet_n contains 4 sets. Each set contains (path/to/ref.xml, path/to/sec.xml)
    
    slc_zips = list of directories containing the topsApp, reference, and secondary .xml files    
    created an returned using the project_dir fucntion
    """
    
    triplet_dict={}
    
    for i in range(len(slc_list) - 2):
        triplet_dict[f'triplet_{i+1}'] = ((slc_list[i],slc_list[i+1]), (slc_list[i+1],slc_list[i+2]), (slc_list[i+2], slc_list[i]))
        if i == 0:
            test_triplet = ((slc_list[i],slc_list[i+1]), (slc_list[i+1],slc_list[i+2]), (slc_list[i+2], slc_list[i]), (slc_list[i], slc_list[i+2]))
        else:
            continue
    return triplet_dict, test_triplet


# SNAP GPT subprocess functions

In [35]:
def split_SLC(sourcefile, subswath, polarization, outfile):
    split_cmd = [
        GPT_PATH,
        "TOPSAR-Split", # operation name
        f"-Ssource={sourcefile[6:]}", # SLC file
        f"-Psubswath={subswath}", #1, 2, 3
        f"-PselectedPolarisations={polarization}", # bands being processed
        "-t", outfile, # keep separate, output file location
        "-q", str(MAX_CORES),
        "-c", MAX_MEM
    ]

    # Run the command
    try:
        result = subprocess.run(split_cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print("Command executed successfully!")
    except subprocess.CalledProcessError as e:
        print(f"Command failed with error code {e.returncode}")
        print(f"Error message: {e.stderr}")

def orbit_SLC(sourcefile, orbittype, outfile):
    orbit_cmd = [
        GPT_PATH,
        "Apply-Orbit-File", # operation name
        f"-Ssource={sourcefile}",
        f"-PorbitType={orbittype}",
        "-t", outfile, # keep separate, output file location
        "-q", str(MAX_CORES),
        "-c", MAX_MEM
    ]

    # Run the command
    try:
        result = subprocess.run(orbit_cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print("Command executed successfully!")
    except subprocess.CalledProcessError as e:
        print(f"Command failed with error code {e.returncode}")
        print(f"Error message: {e.stderr}")

def geocode_SLC(m_file, s_file, demname, resamplemethod, reramp, maskelev, derampdemod, offset, resampletype, outfile):
    geocode_cmd = [
        GPT_PATH,
        "Back-Geocoding", # operation name
        f"-Ssource1={m_file}",
        f"-Ssource2={s_file}",
        f"-PdemName={demname}",
        f"-PdemResamplingMethod={resamplemethod}",
        f"-PdisableReramp={reramp}",
        f"-PmaskOutAreaWithoutElevation={maskelev}",
        f"-PoutputDerampDemodPhase={derampdemod}",
        f"-PoutputRangeAzimuthOffset={offset}",
        f"-PresamplingType={resampletype}",
        "-t", outfile, # keep separate, output file location
        "-q", str(MAX_CORES),
        "-c", MAX_MEM
    ]

    # Run the command
    try:
        result = subprocess.run(geocode_cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print("Command executed successfully!")
    except subprocess.CalledProcessError as e:
        print(f"Command failed with error code {e.returncode}")
        print(f"Error message: {e.stderr}")

def esd_SLC(source_file, cohthresh, writetgt, estimator, winaccazimuth, winaccrange, winheight, winoversampling, winwidth, integrationmethod, maxtempbaseline, blocksperlap, tempbaselinetype, usrazimuthshift, ovrazimuthshift, usrrangeshift, ovrrangeshift, shiftweightfunction, xcorrthersh, outfile):
    esd_cmd = [
        GPT_PATH,
        "Enhanced-Spectral-Diversity", # operation name
        f"-Ssource={source_file}",
        f"-PcohThreshold={cohthresh}",
        f"-PdoNotWriteTargetBands={writetgt}",
        f"-PesdEstimator={estimator}",
        f"-PfineWinAccAzimuth={winaccazimuth}",
        f"-PfineWinAccRange={winaccrange}",
        f"-PfineWinHeightStr={winheight}",
        f"-PfineWinOversampling={winoversampling}",
        f"-PfineWinWidthStr={winwidth}",
        f"-PintegrationMethod={integrationmethod}",
        f"-PmaxTemporalBaseline={maxtempbaseline}",
        f"-PnumBlocksPerOverlap={blocksperlap}",
        f"-PoverallAzimuthShift={ovrazimuthshift}",
        f"-PoverallRangeShift={ovrrangeshift}",
        f"-PtemporalBaselineType={tempbaselinetype}",
        f"-PuseSuppliedAzimuthShift={usrazimuthshift}",
        f"-PuseSuppliedRangeShift={usrrangeshift}",
        f"-PweightFunc={shiftweightfunction}",
        f"-PxCorrThreshold={xcorrthersh}",
        "-t", outfile, # keep separate, output file location
        "-q", str(MAX_CORES),
        "-c", MAX_MEM
    ]

    # Run the command
    try:
        result = subprocess.run(esd_cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print("Command executed successfully!")
    except subprocess.CalledProcessError as e:
        print(f"Command failed with error code {e.returncode}")
        print(f"Error message: {e.stderr}")

def ifgram_SLC(source_file, azicohwin, rgcohwin, demname, eDEMEGM, eDEMfile, eDEMnoval, includecohband, orbitdegree, outputelevationband, outputFEphase, outputLatLon, outputTopophase, cohsquarepixels, FEphasepoints, FEphasepoly, removeFEphase, removeTopophase, demsimulaton, outfile):
    if eDEMfile == 'None':
        ifgram_cmd = [
            GPT_PATH,
            "Interferogram", # operation name
            f"-SsourceProduct={source_file}",
            f"-PcohWinAz={azicohwin}",
            f"-PcohWinRg={rgcohwin}",
            f"-PdemName={demname}",
            f"-PexternalDEMApplyEGM={eDEMEGM}",
            f"-PincludeCoherence={includecohband}",
            f"-PorbitDegree={orbitdegree}",
            f"-PoutputElevation={outputelevationband}",
            f"-PoutputFlatEarthPhase={outputFEphase}",
            f"-PoutputLatLon={outputLatLon}",
            f"-PoutputTopoPhase={outputTopophase}",
            f"-PsquarePixel={cohsquarepixels}",
            f"-PsrpNumberPoints={FEphasepoints}",
            f"-PsrpPolynomialDegree={FEphasepoly}",
            f"-PsubtractFlatEarthPhase={removeFEphase}",
            f"-PsubtractTopographicPhase={removeTopophase}",
            f"-PtileExtensionPercent={demsimulaton}",
            "-t", outfile, # keep separate, output file location
            "-q", str(MAX_CORES),
            "-c", MAX_MEM
        ]
    else:
        ifgram_cmd = [
            GPT_PATH,
            "Interferogram", # operation name
            f"-SsourceProduct={source_file}",
            f"-PcohWinAz={azicohwin}",
            f"-PcohWinRg={rgcohwin}",
            f"-PdemName={demname}",
            f"-PexternalDEMApplyEGM={eDEMEGM}",
            f"-PexternalDEMFile={eDEMfile}",
            f"-PeternalDEMNoDataValue={eDEMnoval}",
            f"-PincludeCoherence={includecohband}",
            f"-PorbitDegree={orbitdegree}",
            f"-PoutputElevation={outputelevationband}",
            f"-PoutputFlatEarthPhase={outputFEphase}",
            f"-PoutputLatLon={outputLatLon}",
            f"-PoutputTopoPhase={outputTopophase}",
            f"-PsquarePixel={cohsquarepixels}",
            f"-PsrpNumberPoints={FEphasepoints}",
            f"-PsrpPolynomialDegree={FEphasepoly}",
            f"-PsubtractFlatEarthPhase={removeFEphase}",
            f"-PsubtractTopographicPhase={removeTopophase}",
            f"-PtileExtensionPercent={demsimulaton}",
            "-t", outfile, # keep separate, output file location
            "-q", str(MAX_CORES),
            "-c", MAX_MEM
        ]

    # Run the command
    try:
        result = subprocess.run(ifgram_cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print("Command executed successfully!")
    except subprocess.CalledProcessError as e:
        print(f"Command failed with error code {e.returncode}")
        print(f"Error message: {e.stderr}")

# Steps?

1. split, orbit, coreg, ESD
2. interferogram (topo phase removal, deburst, filter, subset)
4. snaphu export unwrap import
5. geocode, displacement

# Establish needed paths for notebook

In [4]:
# directory to where the data is stored, this was established in "get_slc.ipynb"
# directory to where 'ASCENDING' and 'DESCENDING' are located so we can process both tracks
# data_dir = '/mnt/d/SabineRS/Sentinel-1'   #WSL
data_dir = '/home/wcc/Desktop/SabineRS/Sentinel-1'  #Linux

asc_dir = os.path.join(data_dir, 'ASCENDING')
des_dir = os.path.join(data_dir, 'DESCENDING')

asc_ini = os.path.join(asc_dir, '0_initial')
des_ini = os.path.join(des_dir, '0_initial')

# Determine the tracks and frames that are stored in the asc_dir and des_dir
- Will use this to form interferograms

In [5]:
# list of ascending track initial orbits and frames found for aoi
display([x[0] for x in os.walk(asc_ini)])

['/home/wcc/Desktop/SabineRS/Sentinel-1/ASCENDING/0_initial',
 '/home/wcc/Desktop/SabineRS/Sentinel-1/ASCENDING/0_initial/136',
 '/home/wcc/Desktop/SabineRS/Sentinel-1/ASCENDING/0_initial/136/95',
 '/home/wcc/Desktop/SabineRS/Sentinel-1/ASCENDING/0_initial/136/91',
 '/home/wcc/Desktop/SabineRS/Sentinel-1/ASCENDING/0_initial/136/93']

In [6]:
# list of ascending track initial orbits and frames found for aoi
display([x[0] for x in os.walk(des_ini)])

[]

# Choose which track-orbit-frame set you want to process based on above outputs
- will work on this later to automatically check number of trackframe pairs, report the number of epochs in each, and then automatically process in order of number of epochs

In [7]:
# for example, if I want to process ascending track 136 frame 93
trackframe = '136/93'
INPUT_DIR = os.path.join(asc_ini, trackframe)  # Directory containing Sentinel-1 SLCs

# Establish pre-processing compute parameters

In [8]:
# get your path to the SNAP GPT install
# typically found in your 'esa-snap' install file in the 'bin' subdirectory
# Linux example: ~/esa-snap/bin/gpt
# WSL example: /mnt/c/Program Files/bin/gpt

# Define paths and parameters
# GPT_PATH = '/mnt/c/Program Files/esa-snap/bin/gpt.exe'  # Path to SNAP's gpt executable, WSL example
GPT_PATH = '/home/wcc/esa-snap/bin/gpt'

MAX_CORES = 20  # Adjust based on system capacity
MAX_MEM = '84G'  # Maximum memory for SNAP, can be K, M, G
TILE_SIZE = 4096  # Tile size for SNAP

# Find all SLC zips for processing

In [9]:
# List all Sentinel-1 SLC files
slc_files = sorted([os.path.join(INPUT_DIR, f) for f in os.listdir(INPUT_DIR) if f.endswith(".zip")])
print("Available Sentinel-1 SLC Files:")
for i, file in enumerate(slc_files):
    print(f"{i + 1}: {file}")

# Select SLC pairs (e.g., manually choose indices for master and slave)
master_index = 0  # Update this to the index of the desired master file
slave_index = 1   # Update this to the index of the desired slave file

master_file = slc_files[master_index]
slave_file = slc_files[slave_index]

print(f"\nSelected Master: {master_file}")
print(f"Selected Slave: {slave_file}")

Available Sentinel-1 SLC Files:
1: /home/wcc/Desktop/SabineRS/Sentinel-1/ASCENDING/0_initial/136/93/S1A_IW_SLC__1SDV_20191001T001837_20191001T001904_029258_035306_76D9.zip
2: /home/wcc/Desktop/SabineRS/Sentinel-1/ASCENDING/0_initial/136/93/S1A_IW_SLC__1SDV_20191013T001837_20191013T001904_029433_035905_4924.zip
3: /home/wcc/Desktop/SabineRS/Sentinel-1/ASCENDING/0_initial/136/93/S1A_IW_SLC__1SDV_20191106T001837_20191106T001904_029783_036530_4CA8.zip
4: /home/wcc/Desktop/SabineRS/Sentinel-1/ASCENDING/0_initial/136/93/S1A_IW_SLC__1SDV_20191118T001837_20191118T001904_029958_036B4F_FD42.zip
5: /home/wcc/Desktop/SabineRS/Sentinel-1/ASCENDING/0_initial/136/93/S1A_IW_SLC__1SDV_20191130T001837_20191130T001904_030133_03715E_A9CE.zip
6: /home/wcc/Desktop/SabineRS/Sentinel-1/ASCENDING/0_initial/136/93/S1A_IW_SLC__1SDV_20191212T001836_20191212T001903_030308_037763_4B09.zip
7: /home/wcc/Desktop/SabineRS/Sentinel-1/ASCENDING/0_initial/136/93/S1A_IW_SLC__1SDV_20200117T001835_20200117T001902_030833_0389

# Form triplet dictionaries needed for non-zero closure phase InSAR

In [10]:
triplet_dict, example_triplet = get_triplets(sorted([os.path.join(INPUT_DIR, f) for f in os.listdir(INPUT_DIR) if f.endswith(".zip")]))
example_triplet

(('/home/wcc/Desktop/SabineRS/Sentinel-1/ASCENDING/0_initial/136/93/S1A_IW_SLC__1SDV_20191001T001837_20191001T001904_029258_035306_76D9.zip',
  '/home/wcc/Desktop/SabineRS/Sentinel-1/ASCENDING/0_initial/136/93/S1A_IW_SLC__1SDV_20191013T001837_20191013T001904_029433_035905_4924.zip'),
 ('/home/wcc/Desktop/SabineRS/Sentinel-1/ASCENDING/0_initial/136/93/S1A_IW_SLC__1SDV_20191013T001837_20191013T001904_029433_035905_4924.zip',
  '/home/wcc/Desktop/SabineRS/Sentinel-1/ASCENDING/0_initial/136/93/S1A_IW_SLC__1SDV_20191106T001837_20191106T001904_029783_036530_4CA8.zip'),
 ('/home/wcc/Desktop/SabineRS/Sentinel-1/ASCENDING/0_initial/136/93/S1A_IW_SLC__1SDV_20191106T001837_20191106T001904_029783_036530_4CA8.zip',
  '/home/wcc/Desktop/SabineRS/Sentinel-1/ASCENDING/0_initial/136/93/S1A_IW_SLC__1SDV_20191001T001837_20191001T001904_029258_035306_76D9.zip'),
 ('/home/wcc/Desktop/SabineRS/Sentinel-1/ASCENDING/0_initial/136/93/S1A_IW_SLC__1SDV_20191001T001837_20191001T001904_029258_035306_76D9.zip',
  '

# Processing steps below come from the MintPy documentation
- https://github.com/insarlab/MintPy/wiki/SNAP-input-data/94265da3b2e6ae4ec420e7a8d6f5e2e937616f85

# 1. TOPSAR-Split

In [ ]:
# Example TOPSAR-Split cell, use this to build more cells for the other steps needed
# may make two notebooks; one for interferograms and the other for the custom backscatter products
# os.chdir('/mnt/d')  #WSL
os.chdir('/home')   #Linux
split_dir = os.path.join(asc_dir, f'1_split/{trackframe}')
os.makedirs(split_dir, exist_ok=True)

subswath = 'IW3'    # 'IW1', 'IW2' or 'IW3'

for i, slc in enumerate(example_triplet[0]):
    if i == 0:
        split_file = os.path.join(split_dir, 'm_testIW3.dim')
    else:
        split_file = os.path.join(split_dir, 's_testIW3.dim')
    
    split_SLC(slc, subswath, 'VV,VH', split_file)

Command executed successfully!
Command executed successfully!


# 2. Apply Orbit Files

In [ ]:
orbit_dir = os.path.join(asc_dir, f'2_orbit/{trackframe}')
os.makedirs(orbit_dir, exist_ok=True)

orbit = 'Sentinel Precise (Auto Download)'  # can change this, refer to the GPT Apply-Orbit-File -h command

for i , file in enumerate([f for f in os.listdir(split_dir) if f.endswith('.dim')]):
    if i == 0:
        orbit_file = os.path.join(orbit_dir, 'm_testIW3.dim')
    else:
        orbit_file = os.path.join(orbit_dir, 's_testIW3.dim')

    orbit_SLC(split_file, orbit, orbit_file)


shutil.rmtree(os.path.join(asc_dir, '1_split')) # removes previous split data, saving storage on your computer 

Command executed successfully!
Command executed successfully!


# 3. Back-Geocoding
- Pass two -Ssource, m first then s

One can pass an external DEM for geocoding, but higher or finer resolution DEMs will not make a difference during geocoding. The external DEM will have an effect during terrain correction steps. Because of this, steps to include the external DEM will only be addressed during terrain correction

In [ ]:
geocode_dir = os.path.join(asc_dir, f'3_geocode/{trackframe}')
os.makedirs(geocode_dir, exist_ok=True)

geocode_file = os.path.join(geocode_dir, 'testIW3.dim')

geocode_SLC([os.path.join(orbit_dir,f) for f in os.listdir(orbit_dir) if f.endswith('.dim')][1],
            [os.path.join(orbit_dir,f) for f in os.listdir(orbit_dir) if f.endswith('.dim')][0],
            'Copernicus 30m Global DEM',
            'CUBIC_CONVOLUTION',    # BICUBIC_INTERPOLATION default; BILINEAR_INTERPOLATION, NEAREST_NEIGHBOR are among others 
            False,                  # boolean, default is False
            True,                   # boolean, default is True
            False,                  # boolean, default is False
            False,                  # boolean, default is False
            'CUBIC_CONVOLUTION',    # BISINC_5_POINT_INTERPOLATION defult; BILINEAR_INTERPOLATION, NEAREST_NEIGHBOR are among others
            geocode_file
            )

shutil.rmtree(os.path.join(asc_dir, '2_orbit')) # removes previous orbit-applied data, saving storage on your computer 

Command executed successfully!


# 4. ESD
- only need this if you have multiple bursts

In [ ]:
esd_dir = os.path.join(asc_dir, f'4_esd/{trackframe}')
os.makedirs(esd_dir, exist_ok=True)

esd_file = os.path.join(esd_dir, 'testIW3.dim')

esd_SLC(geocode_file, 
        0.2,                    # double, (0,1]
        False,                  # boolean, False is default
        'Periodogram',          # 'Periodogram' default, 'Average' is other choice
        '32',                   # string, '16' default, options include '2', '4', '8', '16', '32', '64'
        '32',                   # string, '16' default, options include '2', '4', '8', '16', '32', '64'
        '512',                  # string, '512' default, options include '32', '64', '128', '256', '512', '1024', '2048'
        '128',                  # string, '128' default, options include '32', '64', '128', '256'
        '512',                  # string, '512' default, options include '32', '64', '128', '256', '512', '1024', '2048'
        'L1 and L2',            # 'L1 and L2' default, 'L1' and 'L2' are also options
        0,                      # integer, default 4, <1 generates network with all possible pairs
        10,                     # integer, defaults is 10, [1, 20]
        'Number of images',     # 'Number of images' default, can also be 'Number of days'
        False,                  # boolean, False is default. If True need to adjust user_azimuth_shift
        0.0,                    # double, defult is 0.0. not used if user_user_azimuth_shift=False
        False,                  # boolean, False is default. If True need to adjust user_range_shift
        0.0,                    # double, defult is 0.0. not used if user_user_range_shift=False
        'Inv Quadratic',        # 'Inv Quadratic' default, other options are 'None', 'Linear', 'Quadratic'
        0.1,                    # double, (0, *)
        esd_file)

shutil.rmtree(os.path.join(asc_dir, '3_geocode')) #removes previous back-geocoded data, saving storage on your computer 

Command executed successfully!


# 5. Interferogram formation
- This (or later in Topo Phase Removal Step) is where an external DEM can be used for more accurate results
- Keep in mind, Sentinel-1 SLC pixel resolutions are 2.7 to 3.5 m in range and 22 m in azimuth when choosing external DEM
- EXTERNAL DEM MUST BE IN EPSG:4326

In [ ]:
if_dir = os.path.join(asc_dir, f'5_interferogram/{trackframe}')
os.makedirs(if_dir, exist_ok=True)

if_file = os.path.join(if_dir, 'testIW3.dim')

ifgram_SLC(esd_file, 
           10,                      # integer, default is 10. ASIMUTH 22m/pixel
           10,                      # integer, default is 10. RANGE 2.7 to 3.5m/pixel
           'Copernicus 30m Global DEM',                 # string, DEM name in ESA SNAP
           False,                        # boolean, True is default 
           'None',                    # path to external DEM used for interferogram
           0,                   # double, no data value from external DEM, default is 0
           True,              # boolean, True is default, NEEDED FOR MINTPY
           3,                 # integer, 3 is default, can be 1, 2, 3, 4, or 5
           True,         # boolean, default is False, NEEDED FOR MINTPY
           False,               # boolean, default is False
           False,                # boolean, default is False
           False,             # boolean, Default is False, accomplished later in workflow
           True,             # boolean, default is True, used to make square windows/tiles for coherence estimation. if False, can change first two integers, otherwise can only change coherence window
           601,               # integer, default is 501, number of points used for flat earth polynomial estimation, typically not used
           6,                 # integer, default is 5, [1, 8]
           True,               # boolean, default is True,
           False,             # boolean, default is False. Removes topographic phase delay, will be done in a later step
           '100',                # string of an integer, default is 100
           if_file
           )

shutil.rmtree(os.path.join(asc_dir, '4_esd')) #removes previous esd data, saving storage on your computer 

Command executed successfully!


# 6. TOPSAR-Deburst

In [ ]:
deburst_dir = os.path.join(asc_dir, f'6_deburst/{trackframe}')
os.makedirs(deburst_dir, exist_ok=True)




shutil.rmtree(os.path.join(asc_dir, '5_interferogram')) #removes initial interferogram data, saving storage on your computer 

# 7. Topographic Phase Removal

In [ ]:
Tphase_dir = os.path.join(asc_dir, f'7_topo/{trackframe}')
os.makedirs(Tphase_dir, exist_ok=True)




shutil.rmtree(os.path.join(asc_dir, '6_deburst')) #removes initial interferogram data, saving storage on your computer 

# 8. Goldstein Phase Filtering

In [ ]:
phasefilt_dir = os.path.join(asc_dir, f'8_phasefilt/{trackframe}')
os.makedirs(phasefilt_dir, exist_ok=True)




shutil.rmtree(os.path.join(asc_dir, '7_topo')) #removes initial interferogram data, saving storage on your computer 

# 9. Coherence map generation

In [ ]:
phasefilt_dir = os.path.join(asc_dir, f'8_phasefilt/{trackframe}')
os.makedirs(phasefilt_dir, exist_ok=True)




shutil.rmtree(os.path.join(asc_dir, '7_topo')) #removes initial interferogram data, saving storage on your computer 

# 10. Subset for the MintPy products
Need:
1. Subsetted Interferogram
2. Subsetted elevation band
3. Subsetted coherence band

# 11. Export to SNAPHU for unwrapping